# Load checkpoint and imports

In [1]:
device_name='cpu'

In [2]:
# ! conda remove openpom
import sys
sys.path.remove('/Users/farzaneh/opt/anaconda3/envs/open_pom/lib/python39.zip')

ValueError: list.remove(x): x not in list

In [5]:
import pandas as pd
import ast
import numpy as np
base_path  = '../../../../../../T5 EVO/alignment_olfaction_datasets/data/'
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV
import random
from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import jaccard_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from openpom.utils.data_utils import get_class_imbalance_ratio
import deepchem as dc
import os
import sys
print(sys.path)
# sys.path.append('/Volumes/work/phd/Olfaction/molformer-main')
# sys.path.append('/Volumes/work/phd/Olfaction/molformer-main/openpom/utils')

print(os.getcwd(),"dsss")
import os, sys

# dir_path = os.path.dirname(os.path.realpath(__file__))
# parent_dir_path = os.path.abspath(os.path.join(dir_path, os.pardir))
# sys.path.insert(0, parent_dir_path)
from utils.custom_models  import *


# from sklearn.model_selection import GridSearchCV

# base_directory= '/local_storage/datasets/farzaneh'

['/Volumes/work/phd/Olfaction/molformer-main/notebooks/pretrained_molformer', '/Users/farzaneh/opt/anaconda3/envs/open_pom/lib/python3.9', '/Users/farzaneh/opt/anaconda3/envs/open_pom/lib/python3.9/lib-dynload', '', '/Users/farzaneh/.local/lib/python3.9/site-packages', '/Users/farzaneh/opt/anaconda3/envs/open_pom/lib/python3.9/site-packages', '/Volumes/work/openpom']
/Volumes/work/phd/Olfaction/molformer-main/notebooks/pretrained_molformer dsss


ModuleNotFoundError: No module named 'utils'

In [ ]:
seed = 2024
np.random.seed(seed)
random.seed(seed)
# torch.cuda.manual_seed_all(seed)

# helper methods

In [ ]:
def metrics(y_true,y_predicted_prob):
    y_pred = np.round(y_predicted_prob)
    average_type = [None, 'macro' , 'weighted']

    # for average in average_type:
    ap = average_precision_score(y_true, y_predicted_prob,average=None)
    ap_macro = average_precision_score(y_true, y_predicted_prob,average='macro')
    ap_weighted = average_precision_score(y_true, y_predicted_prob,average='weighted')
    
    precision = precision_score(y_true, y_pred,average=None)
    precision_macro = precision_score(y_true, y_pred,average='macro')
    precision_weighted = precision_score(y_true, y_pred,average='weighted')


    recall = recall_score(y_true, y_pred,average=None)
    recall_macro = recall_score(y_true, y_pred,average='macro')
    recall_weighted = recall_score(y_true, y_pred,average='weighted')


    f1 = f1_score(y_true, y_pred,average=None)
    f1_macro = f1_score(y_true, y_pred,average='macro')
    f1_weighted = f1_score(y_true, y_pred,average='weighted')

    
    # true_labels = np.round(y_true)
    
    # print(true_labels)
    # print(y_true)
    return ap_macro,precision_macro,recall_macro,f1_macro
    

In [ ]:
def custom_logistic_regression(X,y):

    
    logreg = LogisticRegression(max_iter=1000,random_state=seed,n_jobs=-1)
    # base_lr = LogisticRegression()
    ovr = OneVsRestClassifier(logreg)
    estimator=ovr.fit(X, y)
    # estimator= logreg.fit(X,y)
    return estimator

In [ ]:
def pipeline_regression(data_groupbyCID,times,n_components=None,seed=seed):
    mserrorrs_corssvalidated = []
    correlations_corssvalidated = []
    for i in range(times):
        X=np.asarray(data_groupbyCID.embeddings.values.tolist())
        y=np.asarray(data_groupbyCID.y.values.tolist())
        # print("min", X.min())
        # print("max", X.max())
          
        # print("X",X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed+i)  
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        if n_components is not None:
            pca = PCA(n_components=n_components)
            X_train=pca.fit_transform(X_train)
            X_test=pca.transform(X_test)
        linreg =custom_logistic_regression(X_train,y_train)
        mseerrors, correlations=metrics_per_descritor(X_test,y_test,[str(i) for i in range(y.shape[1])],linreg)
        mserrorrs_corssvalidated.append(mseerrors)
        correlations_corssvalidated.append(correlations)
    return mserrorrs_corssvalidated, correlations_corssvalidated

In [ ]:
def mAP(targs, preds, pos_thr=0.5):
    """Returns the model's average precision for each class
    Return:
        ap (FloatTensor): 1xK tensor, with avg precision for each class k
    """
    def average_precision(output, target):
        epsilon = 1e-8

        # sort examples
        indices = output.argsort()[::-1]
        # Computes prec@i
        total_count_ = np.cumsum(np.ones((len(output), 1)))

        target_ = target[indices]
        ind = target_ == 1
        pos_count_ = np.cumsum(ind)
        total = pos_count_[-1]
        pos_count_[np.logical_not(ind)] = 0
        pp = pos_count_ / total_count_
        precision_at_i_ = np.sum(pp)
        precision_at_i = precision_at_i_ / (total + epsilon)

        return precision_at_i

    if np.size(preds) == 0:
        return 0
    ap = np.zeros((preds.shape[1]))
    # compute average precision for each class
    for k in range(preds.shape[1]):
        scores = preds[:, k]
        targets = targs[:, k]
        ap[k] = average_precision(scores, targets)
    tp, fp, fn, tn = [], [], [], []
    conf_matrices = []
    
    for k in range(preds.shape[0]):
        scores = preds[k,:]
        targets = targs[k,:]
        pred = (scores > pos_thr).astype(np.int32)
        tp.append(((pred + targets) == 2).sum())
        fp.append(((pred - targets) == 1).sum())
        fn.append(((pred - targets) == -1).sum())
        tn.append(((pred + targets) == 0).sum())
        conf_matrices.append(confusion_matrix(targets, pred))

    p_c = [tp[i] / (tp[i] + fp[i]) if tp[i] > 0 else 0.0 for i in range(len(tp))]
    r_c = [tp[i] / (tp[i] + fn[i]) if tp[i] > 0 else 0.0
                for i in range(len(tp))]
    f_c = [2 * p_c[i] * r_c[i] / (p_c[i] + r_c[i]) if tp[i] > 0 else 0.0
                for i in range(len(tp))]

    mean_p_c = sum(p_c) / len(p_c)
    mean_r_c = sum(r_c) / len(r_c)
    mean_f_c = sum(f_c) / len(f_c)

    p_o = sum(tp) / (np.array(tp) + np.array(fp)).sum()
    r_o = sum(tp) / (np.array(tp) + np.array(fn)).sum()
    f_o = 2 * p_o * r_o / (p_o + r_o)
    roc_auc=roc_auc_score(targs, preds)

    return ap.mean(), mean_p_c, mean_r_c, mean_f_c, p_o, r_o, f_o,roc_auc, conf_matrices


def tune_multilabel_thresholds(raw_scores, labels):
    scores = 1. / (1. + np.exp(-1. * raw_scores))
    threshs = np.zeros(labels.shape[1])

    for j in range(labels.shape[1]):
        precision, recall, thresholds = precision_recall_curve(labels[:, j], scores[:, j])
        fscore = (2 * precision * recall) / (precision + recall)
        threshs[j] = thresholds[np.argmax(fscore)]

    return threshs

In [ ]:
def run_linear_classifier(modeldeepchem,embedding_dataset,embedding_test_dataset,metric,metric_str,nb_epoch = 2):
    start_time = datetime.now()
    
    train_scores=[]
    valid_scores=[]
    losses=[]
    for epoch in range(1, nb_epoch+1):
        loss = modeldeepchem.fit(
              embedding_dataset,
              nb_epoch=1,
              max_checkpoints_to_keep=1,
              deterministic=False,
              restore=epoch>1)
        losses.append(loss)
    
    modeldeepchem.save_checkpoint()
    end_time = datetime.now()
    return modeldeepchem,train_scores,valid_scores,losses


In [ ]:
def create_linear_classifier(TASKS, n_classes,train_ratios,mode="classification"):
    n_classes = 1
    n_tasks= len(TASKS)
    if mode == "classification":
        model = NonLinearModel(n_tasks).to(device_name)
    elif mode == "classification_small":
        model = LinearModelSmall(n_tasks).to(device_name)
    elif mode == "linear_classification":
        model = LinearModel(n_tasks).to(device_name)
        
    # model.n_tasks=n_tasks
    model.n_classes=1
    
    # ratios = get_class_imbalance_ratio(train_dataset)
    lr = 1e-4
    
    learning_rate = 0.001
    modeldeepchem = LinearModelMultiLabelDeepchem(\
        model=model,class_imbalance_ratio=train_ratios,loss_aggr_type='sum',\
        device_name=device_name,mode=mode,optimizer_name = 'adam',\
        learning_rate=learning_rate,batch_size=128)
    
    return modeldeepchem
    

In [ ]:
def prepare_dataset(ds):
    ds['y'] = ds['y'].apply(ast.literal_eval)
    ds['embeddings'] = ds['embeddings'].apply(ast.literal_eval)
    return ds

In [ ]:
def prepare_dataset_proba(ds):
    ds['y'] = ds['y'].apply(ast.literal_eval)
    ds['proba'] = ds['proba'].apply(ast.literal_eval)
    return ds

In [ ]:
def select_features(input_file):
    ds_alva = pd.read_csv(input_file)


    chemical_features_r=["nCIR",
                     "ZM1", 
                     "GNar", 
                     "S1K", 
                     "piPC08",
                     "MATS1v",
                     "MATS7v",
                     "GATS1v", 
                     "Eig05_AEA(bo)", 
                     "SM02_AEA(bo)",
                     "SM03_AEA(dm)",
                     "SM10_AEA(dm)",
                     "SM13_AEA(dm)",
                      "SpMin3_Bh(v)",
                     "RDF035v",
                     "G1m",
                     "G1v",
                     "G1e",
                     "G3s",
                     "R8u+",
                     "nRCOSR"]

    nonStereoSMILE = list(map(lambda x: "nonStereoSMILES___" + x, chemical_features_r))
    # IsomericSMILES = list(map(lambda x: "IsomericSMILES___" + x, chemical_features_r))
    selected_features = nonStereoSMILE
    features= ['main_idx','nonStereoSMILES']+selected_features
    # print("cc1", ds_alva.columns.values.tolist())
    ds_alva= ds_alva.rename(columns={"Unnamed: 0":"main_idx"})
    # print("cc2", ds_alva.columns.values.tolist())
    ds_alva_selected = ds_alva[features]
    ds_alva_selected = ds_alva_selected.fillna(0)
    ds_alva_selected['embeddings'] = ds_alva_selected[selected_features].values.tolist()
    return ds_alva_selected

In [ ]:
def pipeline(input_file_alva,gs_lf_molformer,times=30,n_components=None,ds="keller",seed=seed):
    # input_file_keller = base_path+'openpom/data/curated_datasets/embeddings/molformer/keller_molformer_embeddings_13_Apr17.csv'
    

    
    keller_alva = select_features(input_file_keller_alva)
    keller_groupbyCID= pd.merge(keller_alva,keller_behavior,on="CID")
    
        
    mserrorrs_keller_corssvalidated, correlations_keller_corssvalidated=pipeline_regression(keller_groupbyCID,times=times,n_components=n_components,seed=seed)
    mserrorrs_mean_keller,mserrorrs_std_keller,correlations_mean_keller,correlations_std_keller =post_process_results(mserrorrs_keller_corssvalidated, correlations_keller_corssvalidated)
    mserrorrs_corssvalidated_keller,statistics_correlations_corssvalidated_keller,pvalues_correlations_corssvalidated_keller=post_process_results_df(mserrorrs_keller_corssvalidated, correlations_keller_corssvalidated)
    df_keller_mse= pd.DataFrame(mserrorrs_corssvalidated_keller)
    df_keller_mse['model'] = model_name
    df_keller_cor= pd.DataFrame(statistics_correlations_corssvalidated_keller)
    df_keller_cor['model'] = model_name
    # df_keller = pd.concat((df_keller_molformer,df_keller_pom))
    return df_keller_mse, df_keller_cor

## Loading MolFormer Model

# Loading Human Embeddings

## Extracting embedding from MoLFormer for GS-LF

In [ ]:
 

gs_lf_tasks = [
'alcoholic', 'aldehydic', 'alliaceous', 'almond', 'amber', 'animal',
'anisic', 'apple', 'apricot', 'aromatic', 'balsamic', 'banana', 'beefy',
'bergamot', 'berry', 'bitter', 'black currant', 'brandy', 'burnt',
'buttery', 'cabbage', 'camphoreous', 'caramellic', 'cedar', 'celery',
'chamomile', 'cheesy', 'cherry', 'chocolate', 'cinnamon', 'citrus', 'clean',
'clove', 'cocoa', 'coconut', 'coffee', 'cognac', 'cooked', 'cooling',
'cortex', 'coumarinic', 'creamy', 'cucumber', 'dairy', 'dry', 'earthy',
'ethereal', 'fatty', 'fermented', 'fishy', 'floral', 'fresh', 'fruit skin',
'fruity', 'garlic', 'gassy', 'geranium', 'grape', 'grapefruit', 'grassy',
'green', 'hawthorn', 'hay', 'hazelnut', 'herbal', 'honey', 'hyacinth',
'jasmin', 'juicy', 'ketonic', 'lactonic', 'lavender', 'leafy', 'leathery',
'lemon', 'lily', 'malty', 'meaty', 'medicinal', 'melon', 'metallic',
'milky', 'mint', 'muguet', 'mushroom', 'musk', 'musty', 'natural', 'nutty',
'odorless', 'oily', 'onion', 'orange', 'orangeflower', 'orris', 'ozone',
'peach', 'pear', 'phenolic', 'pine', 'pineapple', 'plum', 'popcorn',
'potato', 'powdery', 'pungent', 'radish', 'raspberry', 'ripe', 'roasted',
'rose', 'rummy', 'sandalwood', 'savory', 'sharp', 'smoky', 'soapy',
'solvent', 'sour', 'spicy', 'strawberry', 'sulfurous', 'sweaty', 'sweet',
'tea', 'terpenic', 'tobacco', 'tomato', 'tropical', 'vanilla', 'vegetable',
'vetiver', 'violet', 'warm', 'waxy', 'weedy', 'winey', 'woody'
]
# input_file = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/mols_datasets/curated_GS_LF_merged_4983.csv' # or new downloaded file path
# smiles_field = 'nonStereoSMILES'



In [ ]:

input_file_molformer = base_path+'curated_datasets/embeddings/molformer/gslf_molformer_embeddings_13_Apr17.csv'
gs_lf_molformer=pd.read_csv(input_file_molformer)
gs_lf_molformer=prepare_dataset(gs_lf_molformer)
input_file_indices = base_path+'curated_datasets/embeddings/pom/gslf-splits-idx.csv' # or new downloaded file path
indices = pd.read_csv(input_file_indices)
indices_train = indices.loc[indices['split']=='train']['main_idx'].values.tolist()
indices_valid = indices.loc[indices['split']=='valid']['main_idx'].values.tolist()
indices_test = indices.loc[indices['split']=='test']['main_idx'].values.tolist()
gs_lf_molformer_np = np.asarray(gs_lf_molformer.embeddings.tolist())
gs_lf_molformer_y = np.asarray(gs_lf_molformer.y.tolist())
gs_lf_molformer_embeddings_train = gs_lf_molformer_np[indices_train]
gs_lf_molformer_y_train = gs_lf_molformer_y[indices_train]

gs_lf_molformer_embeddings_test = gs_lf_molformer_np[indices_test]
gs_lf_molformer_y_test = gs_lf_molformer_y[indices_test]

gs_lf_molformer_embeddings_valid = gs_lf_molformer_np[indices_valid]
gs_lf_molformer_y_valid = gs_lf_molformer_y[indices_valid]

embedding_train_dataset_gslf = dc.data.DiskDataset.from_numpy(gs_lf_molformer_embeddings_train,gs_lf_molformer_y_train)
embedding_test_dataset_gslf = dc.data.DiskDataset.from_numpy(gs_lf_molformer_embeddings_test,gs_lf_molformer_y_test)
embedding_valid_dataset_gslf = dc.data.DiskDataset.from_numpy(gs_lf_molformer_embeddings_valid,gs_lf_molformer_y_valid)

In [ ]:

input_file_pom = base_path + 'curated_datasets/proba/pom/gslf_pom_proba_Apr24.csv'
gs_lf_pom=pd.read_csv(input_file_pom)
gs_lf_pom=prepare_dataset_proba(gs_lf_pom)
gs_lf_pom.columns.values.tolist()
gs_lf_pom_np = np.asarray(gs_lf_pom.proba.tolist())
gs_lf_pom_y = np.asarray(gs_lf_pom.y.tolist())


gs_lf_pom_proba_train = gs_lf_pom_np[indices_train]
gs_lf_pom_y_train = gs_lf_pom_y[indices_train]

gs_lf_pom_proba_test = gs_lf_pom_np[indices_test]
gs_lf_pom_y_test = gs_lf_pom_y[indices_test]

gs_lf_pom_proba_valid = gs_lf_pom_np[indices_valid]
gs_lf_pom_y_valid = gs_lf_pom_y[indices_valid]



gs_lf_pom_embeddings_train = gs_lf_pom_np[indices_train]
gs_lf_pom_y_train = gs_lf_pom_y[indices_train]

gs_lf_pom_embeddings_test = gs_lf_pom_np[indices_test]
gs_lf_pom_y_test = gs_lf_pom_y[indices_test]

gs_lf_pom_embeddings_valid = gs_lf_pom_np[indices_valid]
gs_lf_pom_y_valid = gs_lf_pom_y[indices_valid]



embedding_train_dataset_gslf_pom = dc.data.DiskDataset.from_numpy(gs_lf_pom_embeddings_train,gs_lf_pom_y_train)
embedding_test_dataset_gslf_pom = dc.data.DiskDataset.from_numpy(gs_lf_pom_embeddings_test,gs_lf_pom_y_test)
embedding_valid_dataset_gslf_pom = dc.data.DiskDataset.from_numpy(gs_lf_pom_embeddings_valid,gs_lf_pom_y_valid)

In [ ]:
# gs_lf_alva.columns.values.tolist()

In [ ]:
input_file_alva= base_path+'curated_datasets/alva/gslf_molecules_alva_17Apr.csv'
# gs_lf_alva=pd.read_csv(input_file_alva)
# gs_lf_alva=pd.read_csv(input_file_alva)

gs_lf_alva=select_features(input_file_alva)
# gs_lf_alva #keep embeddings

gs_lf_y=gs_lf_pom.copy() #only keep y
gs_lf_y.index.names = ['main_idx'] 
# del gs_lf_y['embeddings']
gs_lf_alva=pd.merge(gs_lf_alva, gs_lf_y, on='main_idx')
# gs_lf_alva
# gs_lf_alva.columns.values.tolist()
gs_lf_alva_np = np.asarray(gs_lf_alva.embeddings.tolist())
gs_lf_alva_y = np.asarray(gs_lf_alva.y.tolist())


gs_lf_alva_features_train = gs_lf_alva_np[indices_train]
gs_lf_alva_y_train = gs_lf_alva_y[indices_train]

gs_lf_alva_features_test = gs_lf_alva_np[indices_test]
gs_lf_alva_y_test = gs_lf_alva_y[indices_test]

gs_lf_alva_features_valid = gs_lf_alva_np[indices_valid]
gs_lf_alva_y_valid = gs_lf_alva_y[indices_valid]

# gs_lf_y.columns.

In [ ]:
embedding_train_dataset_gslf_alva = dc.data.DiskDataset.from_numpy(gs_lf_alva_features_train,gs_lf_alva_y_train)
embedding_test_dataset_gslf_alva = dc.data.DiskDataset.from_numpy(gs_lf_alva_features_test,gs_lf_alva_y_test)
embedding_valid_dataset_gslf_alva = dc.data.DiskDataset.from_numpy(gs_lf_alva_features_valid,gs_lf_alva_y_valid)

# Training a Linear Classifier using DeepChem 

##  Training a non-Linear Classifier using DeepChem and Embeddding of GS-LF extracted from MoLFormer

### POM

In [ ]:
mAP(gs_lf_pom_y_test, gs_lf_pom_proba_test, pos_thr=0.5)

### molformer

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_pom)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,"linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf_pom,embedding_test_dataset_gslf_pom,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

In [ ]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

### Alva

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,"classification_small")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

In [ ]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

### POM

In [ ]:
mAP(gs_lf_pom_y_test, gs_lf_pom_proba_test, pos_thr=0.5)

### molformer

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios)
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

In [ ]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

### Alva

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
# modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,"classification_small")
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios)

metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

In [ ]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

In [ ]:
# mAP(gs_lf_alva_y_test, predicted_labels, pos_thr=0.5)

In [ ]:
# estimator= custom_logistic_regression(gs_lf_alva_features_train,gs_lf_alva_y_train)
# predicted_labels=estimator.predict(gs_lf_alva_features_test)

In [ ]:
# mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

##  Training a Linear Classifier using DeepChem and Embeddding of GS-LF extracted from MoLFormer

### POM

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

In [ ]:
mAP(gs_lf_pom_y_test, predicted_labels, pos_thr=0.5)

### molformer

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

In [ ]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

### Alva

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

In [ ]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

### Alva

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
# modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,"classification_small")
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios)

metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

In [ ]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

In [ ]:
# mAP(gs_lf_alva_y_test, predicted_labels, pos_thr=0.5)

In [ ]:
# estimator= custom_logistic_regression(gs_lf_alva_features_train,gs_lf_alva_y_train)
# predicted_labels=estimator.predict(gs_lf_alva_features_test)

In [ ]:
# mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

##  Training a Linear Classifier using DeepChem and Embeddding of GS-LF extracted from MoLFormer

### POM

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

In [ ]:
mAP(gs_lf_pom_y_test, predicted_labels, pos_thr=0.5)

### molformer

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

In [ ]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

### Alva

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

In [ ]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
# modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,"classification_small")
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios)

metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

In [ ]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

In [ ]:
# mAP(gs_lf_alva_y_test, predicted_labels, pos_thr=0.5)

In [ ]:
# estimator= custom_logistic_regression(gs_lf_alva_features_train,gs_lf_alva_y_train)
# predicted_labels=estimator.predict(gs_lf_alva_features_test)

In [ ]:
# mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

##  Training a Linear Classifier using DeepChem and Embeddding of GS-LF extracted from MoLFormer

### POM

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

In [ ]:
mAP(gs_lf_pom_y_test, predicted_labels, pos_thr=0.5)

### molformer

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf)
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf.predict(embedding_test_dataset_gslf)

In [ ]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)

### Alva

In [ ]:
train_ratios = get_class_imbalance_ratio(embedding_train_dataset_gslf_alva)
modeldeepchem_gslf_alva = create_linear_classifier(gs_lf_tasks, 1,train_ratios,mode ="linear_classification")
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)
modeldeepchem_gslf_alva,train_scores_gslf_alva,test_scores_gslf_alva,losses_gslf_alva= run_linear_classifier(modeldeepchem_gslf_alva,embedding_train_dataset_gslf_alva,embedding_test_dataset_gslf_alva,metric_gslf,'roc_auc_score',nb_epoch = 100)
predicted_labels=modeldeepchem_gslf_alva.predict(embedding_test_dataset_gslf_alva)

In [ ]:
mAP(gs_lf_molformer_y_test, predicted_labels, pos_thr=0.5)